# Data Process Utils

This notebook is used to pre-process video clips in data set generated in DataSetUtils. These functions would prepare the data that can feed into the model directly.

## Import Libraries

- numpy
- scipy for generate music spectrum
- os for file system operations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy
import scipy.signal
from os import listdir
from os.path import isfile, join
import moviepy.editor as mpe
import gc
from PIL import Image
import os

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
def extractVideoFrames(v_clip):
    """
    Convert a 5s video clip into 120 frames.
    
    Args:
        v_clip(videoclip):    5s video clip
        
    Returns:
        frames(numpy.array[144*108*3 image]):    collection of frame images
    """
    frames = []
    
    for f in v_clip.iter_frames():
        img = Image.fromarray(f, "RGB")
        img = img.resize((144, 108))
        frames.append(np.asarray(img))
        
    frames = frames[0:120]   # limit to 120 frames
    
    return np.array(frames)
    

In [6]:
def extractAudioFrames(a_clip, sample_rate):
    """
    Convert a 5s audio clip into 120 frames of log-spectrum image.
    
    Args:
        a_clip(audioClip):    5s audio clip
        sample_rate(int):     sample rate of audio source
        
    Returns:
        frames(numpy.array[]):    collection of spectrum of each frame
    
    """
    # TODO: convert stereo audio mono
    # for testing just use left channel
    left = [frame[0] for frame in a_clip.iter_frames()]
    
    # process the whole audio clip to log-spectrogram
    eps=1e-10
    freqs, times, spec = scipy.signal.spectrogram(np.array(left), fs=sample_rate)
    log_specgram = np.log(spec.astype(np.float32) + eps) - np.log(eps)
    
    # split whole spectrogram into 120 frames
    frames = np.array_split(log_specgram, 120, axis=1)
    
    length = frames[0].shape[1]
    for i in range(len(frames)):
        # np.array_split would make some arraies 1 length shorter that other.
        # pad those arraies using the value from their last column
        if (frames[i].shape[1] != length):
            frames[i] = np.pad(frames[i], ((0, 0), (0, 1)), 'reflect')
    
    return np.array(frames)

In [7]:
def shape_eq(shape1, shape2):
    if (len(shape1)) != (len(shape2)):
        return False
    for i in range(len(shape1)):
        # print(str(shape1[i]) + "  " + str(shape2[i]))
        if (shape1[i] != shape2[i]):
            return False
    return True

## Read data set

Main function in this notebook, readin all clips from a directory and generate frame arraies for video and audio of each clip.

In [8]:
def readDataSet(clips_dir, sample_rate, num = 2):
    """
    Read all clips from a given directory. Convert clips into frames.
    
    Args:
        clips_dir(string):    directory that contains all 5s-vedio clips
        sample_rate(int):     sample rate of audio source, such as 48kHz
        num(int):             number of clips to load
        
    Returns:
        data_set([name, video_clip, audio_clip]):       2D array contains information about each clip
    """
    data_set = []
    
    # get list of filenames in that directory
    files = [f for f in listdir(clips_dir) if isfile(join(clips_dir, f))]
    
    
    counter = num
    for f in files:
        print("----- executing ------")
        print(f)
        v_clip = mpe.VideoFileClip(clips_dir + "/" + f, fps_source="fps", audio_fps=sample_rate)
        a_clip = v_clip.audio
        row = [f[:-4], extractVideoFrames(v_clip), extractAudioFrames(a_clip, sample_rate)]
        # print(row[1].shape)
        # print(row[2].shape)
        if shape_eq(row[1].shape, (120, 108, 144, 3)) and shape_eq(row[2].shape, (120, 129, 9)):
            data_set.append(row)
            print("----- succeeded -------")
            counter -= 1
            if counter ==0:
                break
        else:
            print("----- invalid clip -------")
            if os.path.isfile(clips_dir + "/" + f):
                os.remove(clips_dir + "/" + f)
                print("----- deleted -------")
    
    return data_set

In [12]:
def process_and_save_data_set(clips_dir, data_dir, set_name, sample_rate = 48000, num = 2):
    print("----- start " + set_name + " -------")
    print("loading from " + clips_dir + " ...")
    data = readDataSet(clips_dir, sample_rate, num)
    
    videos = np.asarray([d[1] for d in data])
    audios = np.asarray([d[2] for d in data])
    
    print("storing to " + data_dir + " ...")
    np.save(data_dir + "/" + set_name + "_videos", videos)
    np.save(data_dir + "/" + set_name + "_audios", audios)
    print("----- succeeded -------")

In [13]:
def load_data(data_dir, set_name):
    videos = np.load(data_dir + "/" + set_name + "_videos.npy")
    audios = np.load(data_dir + "/" + set_name + "_audios.npy")
    
    return videos, audios

In [ ]:
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_168", 48000, 168)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_296", 48000, 296)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_612", 48000, 612)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_1224", 48000, 1224)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_2448", 48000, 2448)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_4896", 48000, 4896)
process_and_save_data_set("../data/5s_clips", "../data/numpy/5s_clips", "audioSet_7200", 48000, 7200)
process_and_save_data_set("../data/5s_clips_dance", "../data/numpy/5s_clips", "danceSet_140", 48000, 140)
process_and_save_data_set("../data/5s_clips_dance", "../data/numpy/5s_clips", "danceSet_240", 48000, 240)

----- start audioSet_168 -------
loading from ../data/5s_clips ...
----- executing ------
001253.mp4
----- succeeded -------
----- executing ------
004693.mp4
----- succeeded -------
----- executing ------
005155.mp4
----- succeeded -------
----- executing ------
002493.mp4
----- succeeded -------
----- executing ------
001683.mp4
----- succeeded -------
----- executing ------
000228.mp4
----- succeeded -------
----- executing ------
006650.mp4
----- succeeded -------
----- executing ------
005731.mp4
----- succeeded -------
----- executing ------
005704.mp4
----- succeeded -------
----- executing ------
004655.mp4
----- succeeded -------
----- executing ------
008351.mp4
----- succeeded -------
----- executing ------
000502.mp4
----- succeeded -------
----- executing ------
003609.mp4
----- succeeded -------
----- executing ------
003201.mp4
----- succeeded -------
----- executing ------
000439.mp4
----- succeeded -------
----- executing ------
001634.mp4
----- succeeded -------
-----

----- succeeded -------
----- executing ------
007192.mp4
----- succeeded -------
----- executing ------
002155.mp4
----- succeeded -------
----- executing ------
003874.mp4
----- succeeded -------
----- executing ------
006341.mp4
----- succeeded -------
----- executing ------
004115.mp4
----- succeeded -------
----- executing ------
005108.mp4
----- succeeded -------
----- executing ------
004175.mp4
----- succeeded -------
----- executing ------
001226.mp4
----- succeeded -------
----- executing ------
004593.mp4
----- succeeded -------
----- executing ------
007160.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
004145.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
007047.mp4
----- succeeded -------
----- executing ------
003812.mp4
----- succeeded -------
----- executing ------
007641.mp4
----- succeeded -------
----- executing ------
005175.mp4
----- succeeded -------
----- executing ------
002341.mp4
----- succeeded ------

----- succeeded -------
----- executing ------
003688.mp4
----- succeeded -------
----- executing ------
006130.mp4
----- succeeded -------
----- executing ------
003044.mp4
----- succeeded -------
----- executing ------
007786.mp4
----- succeeded -------
----- executing ------
005665.mp4
----- succeeded -------
----- executing ------
004548.mp4
----- succeeded -------
----- executing ------
000875.mp4
----- succeeded -------
----- executing ------
004730.mp4
----- succeeded -------
----- executing ------
005803.mp4
----- succeeded -------
----- executing ------
007092.mp4
----- succeeded -------
----- executing ------
004820.mp4
----- succeeded -------
----- executing ------
002146.mp4
----- succeeded -------
----- executing ------
001003.mp4
----- succeeded -------
----- executing ------
002304.mp4
----- succeeded -------
----- executing ------
004751.mp4
----- succeeded -------
----- executing ------
001897.mp4
----- succeeded -------
----- executing ------
006649.mp4
----- succeede

----- succeeded -------
----- executing ------
005504.mp4
----- succeeded -------
----- executing ------
005756.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
008163.mp4
----- succeeded -------
----- executing ------
000991.mp4
----- succeeded -------
----- executing ------
007229.mp4
----- succeeded -------
----- executing ------
008459.mp4
----- succeeded -------
----- executing ------
003936.mp4
----- succeeded -------
----- executing ------
008458.mp4
----- succeeded -------
----- executing ------
004099.mp4
----- succeeded -------
----- executing ------
001150.mp4
----- succeeded -------
----- executing ------
003345.mp4
----- succeeded -------
----- executing ------
006918.mp4
----- succeeded -------
----- executing ------
007209.mp4
----- succeeded -------
----- executing ------
004779.mp4
----- succeeded -------
----- executing ------
005434.mp4
----- succeeded -------
----- executing ------
006493.mp4
----- succeeded -------
----- executing ------


----- succeeded -------
----- executing ------
005989.mp4
----- succeeded -------
----- executing ------
005727.mp4
----- succeeded -------
----- executing ------
003678.mp4
----- succeeded -------
----- executing ------
003966.mp4
----- succeeded -------
----- executing ------
005069.mp4
----- succeeded -------
----- executing ------
003982.mp4
----- succeeded -------
----- executing ------
005169.mp4
----- succeeded -------
----- executing ------
007647.mp4
----- succeeded -------
----- executing ------
000213.mp4
----- succeeded -------
----- executing ------
000222.mp4
----- succeeded -------
----- executing ------
006421.mp4
----- succeeded -------
----- executing ------
006641.mp4
----- succeeded -------
----- executing ------
005960.mp4
----- succeeded -------
----- executing ------
002562.mp4
----- succeeded -------
----- executing ------
005965.mp4
----- succeeded -------
----- executing ------
004761.mp4
----- succeeded -------
----- executing ------
001367.mp4
----- succeede

...

----- succeeded -------
----- executing ------
006171.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
005078.mp4
----- succeeded -------
----- executing ------
001653.mp4
----- succeeded -------
----- executing ------
003946.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
000076.mp4
----- succeeded -------
----- executing ------
007975.mp4
----- succeeded -------
----- executing ------
002612.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
000447.mp4
----- succeeded -------
----- executing ------
001825.mp4
----- succeeded -------
----- executing ------
001407.mp4
----- succeeded -------
----- executing ------
003601.mp4
----- succeeded -------
----- executing ------
002473.mp4
----- succeeded -------
----- executing ------
006152.mp4
----- succeeded -------
----- executing ------
008585.mp4
----- succeeded -------
----- executing ------
004698.mp4
----- succeeded -------
----- executing ------
001964.m

----- succeeded -------
----- executing ------
000922.mp4
----- succeeded -------
----- executing ------
004192.mp4
----- succeeded -------
----- executing ------
006177.mp4
----- succeeded -------
----- executing ------
008294.mp4
----- succeeded -------
----- executing ------
000303.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
002932.mp4
----- succeeded -------
----- executing ------
001833.mp4
----- succeeded -------
----- executing ------
008462.mp4
----- succeeded -------
----- executing ------
007991.mp4
----- succeeded -------
----- executing ------
005643.mp4
----- succeeded -------
----- executing ------
003509.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
006710.mp4
----- succeeded -------
----- executing ------
007381.mp4
----- succeeded -------
----- executing ------
002017.mp4
----- succeeded -------
----- executing ------
003107.mp4
----- succeeded -------
----- executing ------
005483.mp4
----- succeeded ------

----- succeeded -------
----- executing ------
008536.mp4
----- succeeded -------
----- executing ------
003133.mp4
----- succeeded -------
----- executing ------
003206.mp4
----- succeeded -------
----- executing ------
005634.mp4
----- succeeded -------
----- executing ------
008060.mp4
----- succeeded -------
----- executing ------
003149.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
006494.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
005292.mp4
----- succeeded -------
----- executing ------
001543.mp4
----- succeeded -------
----- executing ------
002483.mp4
----- succeeded -------
----- executing ------
006486.mp4
----- succeeded -------
----- executing ------
003873.mp4
----- succeeded -------
----- executing ------
003479.mp4
----- succeeded -------
----- executing ------
001629.mp4
----- succeeded -------
----- executing ------
008761.mp4
----- succeeded -------
----- executing ------
008588.mp4
----- succeeded ------

----- invalid clip -------
----- deleted -------
----- executing ------
002031.mp4
----- succeeded -------
----- executing ------
000743.mp4
----- succeeded -------
----- executing ------
003529.mp4
----- succeeded -------
----- executing ------
001167.mp4
----- succeeded -------
----- executing ------
001183.mp4
----- succeeded -------
----- executing ------
005481.mp4
----- succeeded -------
----- executing ------
005777.mp4
----- succeeded -------
----- executing ------
003083.mp4
----- succeeded -------
----- executing ------
001261.mp4
----- succeeded -------
----- executing ------
004294.mp4
----- succeeded -------
----- executing ------
004449.mp4
----- succeeded -------
----- executing ------
004363.mp4
----- succeeded -------
----- executing ------
006881.mp4
----- succeeded -------
----- executing ------
005969.mp4
----- succeeded -------
----- executing ------
002574.mp4
----- succeeded -------
----- executing ------
000147.mp4
----- succeeded -------
----- executing ------


----- succeeded -------
----- executing ------
000459.mp4
----- succeeded -------
----- executing ------
003841.mp4
----- succeeded -------
----- executing ------
001758.mp4
----- succeeded -------
----- executing ------
005973.mp4
----- succeeded -------
----- executing ------
001826.mp4
----- succeeded -------
----- executing ------
000049.mp4
----- succeeded -------
----- executing ------
002441.mp4
----- succeeded -------
----- executing ------
006936.mp4
----- succeeded -------
----- executing ------
002765.mp4
----- succeeded -------
----- executing ------
003577.mp4
----- invalid clip -------
----- deleted -------
----- executing ------
001301.mp4
----- succeeded -------
----- executing ------
003328.mp4
----- succeeded -------
----- executing ------
003631.mp4
----- succeeded -------
----- executing ------
007680.mp4
----- succeeded -------
----- executing ------
005241.mp4
----- succeeded -------
----- executing ------
006284.mp4
----- succeeded -------
----- executing ------


In [58]:
# print(len(data))
# print(data[0][1].shape)
# print(data[0][2].shape)

In [59]:
# d = [d[1] for d in data]

In [13]:
# d[0].shape